In [15]:
# draw ensemble analys
# id,arrival_time,start_service_times,start_service_times,completion_time
# time,event_id,event_type,timing,length_event_list,length_queue1,length_queue2,in_service1,in_service2

import os

In [13]:
# get list of file path based on Parameters:
#mean_interarrival,mean_machine_time_1,mean_machine_time_2,max_queue_1,max_queue_2,time_units,std_machine_1
from dataclasses import dataclass, field

@dataclass
class Parameter:
    mean_interarrival:float
    mean_machine_time_1:float
    mean_machine_time_2:float
    max_queue_1:int
    max_queue_2:int
    time_units:str
    std_machine_1:float

def get_parameter(s):
    s = s.split(',')
    
    return Parameter(
        mean_interarrival=float(s[0]),
        mean_machine_time_1=float(s[1]),
        mean_machine_time_2=float(s[2]),
        max_queue_1=int(s[3]),
        max_queue_2=int(s[4]),
        time_units=s[5],
        std_machine_1=float(s[6])
    )

# the file path should look like this: data/mean_interarrival_60.0/max_queue_1_9223372036854775807/max_queue_2_4/mean_machine_time_125.0
def get_file_path_from_parameters(p, n):
    states_files, entities_files= [],[]
    root_path = f"data/mean_interarrival_{p.mean_interarrival}/max_queue_1_{p.max_queue_1}/max_queue_2_{p.max_queue_2}/mean_machine_time_1{p.mean_machine_time_1}/mean_machine_time_2{p.mean_machine_time_2}"
    for i in range(1,n+1):
        states_files.append(f"{root_path}/seed{i}/state.csv")
        entities_files.append(f"{root_path}/seed{i}/entities.csv")
    return states_files, entities_files


In [27]:
p = get_parameter("60.0, 25.0, 59.0, 9223372036854775807,4, \"minutes\", 7.185832891717325")
states_file_path , entities_file_path = get_file_path_from_parameters(p,10)

In [38]:
# read all states files and get ensemble data

import pandas as pd
import numpy as np

def get_ensemble_data(states_file_path):
    states = []
    for file_path in states_file_path[:2]:
        states.append(pd.read_csv(file_path, skiprows=9))
    # create group the data base on "time" column with time step of 10, filter on timing == "before"
    ensemble_states = [s[(s["timing"] == "before") & (s["event_type"] == "Arrival")] for s in states]    # drop the event_type and timing columns
    ensemble_states = [s.drop(columns=["event_type", "timing"]) for s in ensemble_states]
    ensemble_states = [s.groupby(np.arange(len(s))//10).mean() for s in ensemble_states]

    return states, ensemble_states

states, ensemble_states = get_ensemble_data(states_file_path)


In [39]:
ensemble_states

[Empty DataFrame
 Columns: [time, event_id, length_event_list, length_queue1, length_queue2, in_service1, in_service2]
 Index: [],
 Empty DataFrame
 Columns: [time, event_id, length_event_list, length_queue1, length_queue2, in_service1, in_service2]
 Index: [],
 Empty DataFrame
 Columns: [time, event_id, length_event_list, length_queue1, length_queue2, in_service1, in_service2]
 Index: [],
 Empty DataFrame
 Columns: [time, event_id, length_event_list, length_queue1, length_queue2, in_service1, in_service2]
 Index: [],
 Empty DataFrame
 Columns: [time, event_id, length_event_list, length_queue1, length_queue2, in_service1, in_service2]
 Index: [],
 Empty DataFrame
 Columns: [time, event_id, length_event_list, length_queue1, length_queue2, in_service1, in_service2]
 Index: [],
 Empty DataFrame
 Columns: [time, event_id, length_event_list, length_queue1, length_queue2, in_service1, in_service2]
 Index: [],
 Empty DataFrame
 Columns: [time, event_id, length_event_list, length_queue1, lengt